# Training

## Libraries

In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
from glob import glob
import os

## Dataset loading

In [7]:
'''
Dataset structure:
audio/
├── train
│  ├── class
│  │   ├── 1.wav
│  │   ├── 2.wav
│  │   ├── ...
│  └── ...
'''
def load_dataset(type="train"):
    # Path to audio folder
    audio_folder = "audio/{}".format(type)
    # Extract all classes available by reading the subfolder names
    classes = sorted(os.listdir(audio_folder))
    # Extract all audio files available for each class as a separate list
    audio_files = {}
    for c in classes:
        # Get all files in the class folder
        audio_files[c] = sorted(glob(os.path.join(audio_folder, c, "*.wav")))

    print("Classes: ", classes)
    print(audio_files)
    return audio_files, classes

# load_dataset("train")


## Feature extractions

In [281]:
def zcr(frame, frame_length):
    """Compute Zero Crossing Rate (ZCR)"""
    # Count the number of times the signal changes sign
    # zero_crossings = np.sum(np.abs(np.diff(np.sign(frame)))) / 2
    # return zero_crossings / frame_length
    return librosa.feature.zero_crossing_rate(frame, frame_length=frame_length)[0, 0]

def rms(frame):
    """Compute Root Mean Square (RMS)"""
    # RMS measures the average power of the signal
    # return np.sqrt(np.sum(frame**2) / len(frame))
    return np.sqrt(np.mean(frame**2))

def temporal_entropy(frame):
    """Compute Temporal Entropy"""
    # Temporal entropy measures the distribution of energy in the time domain
    hist = np.histogram(frame, bins=8, range=(np.min(frame), np.max(frame)))[0]
    prob = hist / np.sum(hist)
    prob = prob[prob > 0]  # Avoid log(0)
    return -np.sum(prob * np.log2(prob))

def compute_fft(y, frame_length, hop_length):
    """Compute the Short-Time Fourier Transform (STFT) using NumPy."""
    # Frame-based processing
    # frames = librosa.util.frame(y, frame_length=frame_length, hop_length=hop_length)
    # window = np.hanning(frame_length)  # Apply a Hanning window
    # fft_result = np.fft.rfft(frames * window[:, None], axis=0)  # Compute FFT
    # return np.abs(fft_result)  # Return the magnitude spectrum
    return librosa.stft(y, n_fft=frame_length, hop_length=hop_length, window='hann', center=False)

def spectral_centroid(S, sr):
    """Compute Spectral Centroid"""
    # Spectral centroid is the weighted mean of the frequencies
    # freqs = np.fft.rfftfreq(S.shape[0] * 2 - 1, d=1/sr)
    # magnitude = np.sum(S, axis=1)
    # centroid = np.sum(freqs * magnitude) / np.sum(magnitude)
    # return centroid
    return librosa.feature.spectral_centroid(S=S, sr=sr)

def spectral_rolloff(S, sr, roll_percent=0.85):
    """Compute Spectral Rolloff"""
    # Spectral rolloff is the frequency below which a certain percentage of the total spectral energy is contained
    # freqs = np.fft.rfftfreq(S.shape[0] * 2 - 1, d=1/sr)
    # total_energy = np.sum(S)
    # cumulative_energy = np.cumsum(S)
    # rolloff_idx = np.where(cumulative_energy >= roll_percent * total_energy)[0]
    # if len(rolloff_idx) == 0:  # Handle case where no index satisfies the condition
    #     return freqs[-1]  # Return the highest frequency
    # return freqs[rolloff_idx[0]]
    # Using librosa's built-in function for simplicity
    return librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=roll_percent)

def spectral_flatness(S):
    """Compute Spectral Flatness"""
    # Spectral flatness is the ratio of the geometric mean to the arithmetic mean of the spectrum
    # geometric_mean = np.exp(np.mean(np.log(S + 1e-10)))  # Add small value to avoid log(0)
    # arithmetic_mean = np.mean(S)
    # return geometric_mean / arithmetic_mean
    return librosa.feature.spectral_flatness(S=S)

def band_ratio(S, sr, frame_length):
    """Compute Band Energy Ratio (low vs mid frequencies)"""
    # Band energy ratio compares the energy in different frequency bands
    # freqs = np.fft.rfftfreq(frame_length, d=1/sr)
    freqs = librosa.fft_frequencies(sr=sr, n_fft=frame_length)
    low_band = (freqs >= 100) & (freqs < 1000)
    mid_band = (freqs >= 1000) & (freqs < 4000)
    low_energy = np.sum(S[low_band, :], axis=0)
    mid_energy = np.sum(S[mid_band, :], axis=0)
    return mid_energy / (low_energy + 1e-10)
    # Add small value to avoid division by zero

In [276]:
def extract_audio_features(file_path, sr, frame_length=2048, hop_length=512):
    """Extract material sound features compatible with HDC requirements"""
    
    # Load audio with optimal parameters for material sounds
    y, sr = librosa.load(file_path, sr=sr, duration=5.0)  # 16kHz sampling
    
    # Frame-based processing
    frames = librosa.util.frame(y, frame_length=frame_length, hop_length=hop_length)
    num_frames = frames.shape[1]
    
    # Initialize feature arrays with zeros
    feature_names = ['zcr', 'rms', 'temporal_entropy', 'spectral_centroid', 'spectral_rolloff', 'spectral_flatness', 'band_ratio']
    features = {name: np.zeros(num_frames) for name in feature_names}

    # Time-domain features
    for i in range(num_frames):
        frame = frames[:, i]
        features['zcr'][i] = zcr(frame, frame_length)
        features['rms'][i] = rms(frame)
        features['temporal_entropy'][i] = temporal_entropy(frame)

    # Frequency-domain features
    S = np.abs(librosa.stft(y, n_fft=frame_length, hop_length=hop_length))
    features['spectral_centroid'] = spectral_centroid(S, sr)
    features['spectral_rolloff'] = spectral_rolloff(S, sr)
    features['spectral_flatness'] = spectral_flatness(S)

    # Band energy ratio (low vs mid frequencies)
    features['band_ratio'] = band_ratio(S, sr, frame_length)

    # Aggregate statistics for HDC encoding
    feature_vector = [
        np.mean(features['zcr']), np.std(features['zcr']),
        np.mean(features['rms']), np.max(features['rms']),
        np.mean(features['temporal_entropy']),
        np.mean(features['spectral_centroid']),
        np.mean(features['spectral_rolloff']),
        np.mean(features['spectral_flatness']),
        np.mean(features['band_ratio'])
    ]
    
    return feature_vector

## Data preprocessing

In [ ]:
# # Extract features for all audio files and encode them
# dataset = []
# labels = []

# audio_files, classes = load_dataset("train")


# for label_idx, class_name in enumerate(classes):
#     for file_path in audio_files[class_name]:
#         # Extract features
#         feature_vector = extract_audio_features(file_path, sr=8000)
#         dataset.append(feature_vector)
#         labels.append(label_idx)

# # normailize the dataset where each column is a feature
# dataset = np.array(dataset) 
# dataset = (dataset - np.min(dataset, axis=0)) / (np.max(dataset, axis=0) - np.min(dataset, axis=0))

# # visualize the dataset in heatmap
# plt.figure(figsize=(10, 8))
# plt.imshow(dataset, aspect='auto', cmap='hot')
# plt.colorbar()

## HDC operations

In [277]:
# --- Feature Setup ---
feature_names = [
    'zcr_mean', 'zcr_std', 'rms_mean', 'rms_max',
    'entropy_mean', 'spectral_centroid_mean', 'spectral_rolloff_mean',
    'spectral_flatness_mean', 'band_ratio_mean'
]

# Selectively chosen important feature pairs
important_pairs = [
    ('zcr_mean', 'entropy_mean'),
    ('rms_mean', 'spectral_rolloff_mean'),
    ('spectral_flatness_mean', 'spectral_centroid_mean'),
    ('rms_max', 'band_ratio_mean')
]

# --- 1. Feature name codebook (via permutation) ---
def generate_feature_codebook(feature_names, D):
    base = np.random.randint(0, 2, D, dtype=np.uint8)
    return {name: np.roll(base, i + 1) for i, name in enumerate(feature_names)}

# --- 2. Pre-generate value level hypervectors ---
def generate_value_level_hvs(levels, D):
    level_hvs = []
    for level in range(levels):
        hv = np.zeros(D, dtype=np.uint8)
        if level > 0:
            n_bits = level * D // levels
            indices = np.random.choice(D, n_bits, replace=False)
            hv[indices] = 1
        level_hvs.append(hv)
    return level_hvs

# --- 3. Map value to nearest level HV ---
def get_value_hv(levels, value, level_hvs):
    level = min(levels - 1, max(0, int(value * levels)))
    return level_hvs[level]

# --- 4. Encode single audio feature vector ---
def encode_feature_vector(features, codebook, level_hvs, D, levels):
    assert len(features) == len(codebook)
    
    feature_dict = dict(zip(codebook.keys(), features))
    hvs = []

    # Step 1: Encode individual features (key ⊙ value)
    for name, value in feature_dict.items():
        feat_hv = np.bitwise_xor(codebook[name], get_value_hv(levels, value, level_hvs))
        hvs.append(feat_hv.astype(np.int16))

    # Step 2: Encode selected feature-pair interactions (bound pair HVs)
    for f1, f2 in important_pairs:
        hv1 = np.bitwise_xor(codebook[f1], get_value_hv(levels, feature_dict[f1], level_hvs))
        hv2 = np.bitwise_xor(codebook[f2], get_value_hv(levels, feature_dict[f2], level_hvs))
        pair_hv = np.bitwise_xor(hv1, hv2)
        hvs.append(pair_hv.astype(np.int16))

    # Optional: visualize before bundling
    # plt.figure(figsize=(10, 8))
    # plt.imshow(hvs, aspect='auto', cmap='hot', vmin=0, vmax=1)
    # plt.colorbar()
    
    # Step 3: Final bundling (majority vote)
    hvs = np.array(hvs)
    sum_hv = np.sum(hvs, axis=0)
    threshold = len(hvs) // 2
    final_hv = (sum_hv > threshold).astype(np.uint8)
    # print(sum_hv.shape, hvs.shape, len(hvs),final_hv.shape)

    return final_hv

#######


In [96]:
# make array of all values in the codebook
codebook_values = np.array(list(codebook.values()))
plt.figure(figsize=(12, 10))
plt.imshow(vectors, aspect='auto', cmap=plt.cm.gray) #, vmin=0, vmax=1)
plt.colorbar()


## Model training

In [278]:
def train_hd_classifier(dataset, labels, codebook, level_hvs, D, levels, epochs):
    """
    dataset: list or np.array of normalized feature vectors (N x 9)
    labels: list or np.array of corresponding class labels (N)
    codebook: feature-name -> HVs (symbolic keys of 9 features)
    level_hvs: list of pre-generated value level HVs
    """

    dataset = np.copy(dataset)
    labels = np.copy(labels)

    num_classes = len(np.unique(labels))
    real_class_hvs = np.zeros((num_classes, D), dtype=np.int16)

    N = len(dataset)

    for epoch in range(epochs):
        for i in range(N):
            query_hv = dataset[i]
            y_true = labels[i]

            # Binarize class HVs
            bin_class_hvs = (real_class_hvs >= 0).astype(np.uint8)

            # Predict using Hamming distance
            predictions = np.sum(query_hv != bin_class_hvs, axis=1)
            y_pred = np.argmin(predictions)

            # OnlineHD-style update
            if y_pred != y_true:
                real_class_hvs[y_true] += query_hv
                real_class_hvs[y_pred] -= query_hv

        # Shuffle
        indices = np.random.permutation(N)
        dataset = dataset[indices]
        labels = labels[indices]

    final_class_hvs = np.zeros((num_classes, D), dtype=np.int16)
    final_class_hvs = (real_class_hvs >= 0).astype(np.uint8)
    return final_class_hvs

def predict_hd(query_hv, class_hvs):
    """
    Predict label for a query hypervector using Hamming distance.
    """
    distances = [np.sum(query_hv != class_hv) for class_hv in class_hvs]
    return np.argmin(distances)

def evaluate_hd(vectors, labels, class_hvs):
    correct = 0
    for query_hv, y_true in zip(vectors, labels, strict=True):

        # query_hv = encode_feature_vector(x, codebook, level_hvs, D, levels)

        y_pred = predict_hd(query_hv, class_hvs)
        if y_pred == y_true:
            correct += 1
        # else:
            # print(f"Predicted: {y_pred}, True: {y_true}")
    return correct / len(labels)

# def evaluate_hd(dataset, labels, label_to_id, id_to_label, class_hvs, codebook, level_hvs, D, levels):
#     correct = 0
#     for x, y_true in zip(dataset, labels):
#         query_hv = encode_feature_vector(x, codebook, level_hvs, D, levels)
#         y_pred = predict_hd(query_hv, class_hvs)
#         if y_pred == y_true:
#             correct += 1
#         else:
#             print(f"Predicted: {y_pred}, True: {y_true}")
#     return correct / len(labels)



In [279]:
# --- Main execution ---

# Load dataset
dataset = [] # audio features
labels = [] # class ids

audio_files, classes = load_dataset("train")

# Prepare a dictionary of labels and their IDs
label_to_id = {class_name: idx for idx, class_name in enumerate(classes)}
id_to_label = {idx: class_name for class_name, idx in label_to_id.items()}

# Extract features
for _, class_name in enumerate(classes):
    for file_path in audio_files[class_name]:
        features = extract_audio_features(file_path, sr=8000)
        dataset.append(features)
        labels.append(label_to_id[class_name])


# Normalize feature-wise (0–1 scaling)
dataset = np.array(dataset)
dataset = (dataset - dataset.min(axis=0)) / (dataset.max(axis=0) - dataset.min(axis=0))

# generate codebooks
D = 10000  # Hypervector dimensionality
LEVELS = 256 # quantization levels
np.random.seed(42)
codebook = generate_feature_codebook(feature_names, D)
value_level_hvs = generate_value_level_hvs(LEVELS, D)

vectors = [] # encode dataset vectors

for features in dataset:
    encoded_hv = encode_feature_vector(features, codebook, value_level_hvs, D, LEVELS)
    vectors.append(encoded_hv)

vectors = np.array(vectors)
labels = np.array(labels)

for i in range(20):
    # train the classifier
    class_hvs = train_hd_classifier(vectors, labels, codebook, value_level_hvs, D, LEVELS, epochs=i+1)

    # # Save the trained model
    np.save("class_hvs.npy", class_hvs) # save class hypervectors
    np.save("codebook.npy", codebook) # save codebook
    np.save("value_level_hvs.npy", value_level_hvs) # save value level hypervectors
    np.save("label_to_id.npy", label_to_id) # save label_to_id mapping

    acc = evaluate_hd(vectors, labels, class_hvs)
    print(f"{i+1}: Training accuracy: {acc * 100:.2f}%")



Classes:  ['bell', 'glass', 'saw']
{'bell': ['audio/train/bell/1.wav', 'audio/train/bell/2.wav', 'audio/train/bell/3.wav', 'audio/train/bell/4.wav'], 'glass': ['audio/train/glass/1.wav', 'audio/train/glass/2.wav', 'audio/train/glass/3.wav', 'audio/train/glass/4.wav'], 'saw': ['audio/train/saw/1.wav', 'audio/train/saw/2.wav', 'audio/train/saw/3.wav', 'audio/train/saw/4.wav']}
1: Training accuracy: 58.33%
2: Training accuracy: 91.67%
3: Training accuracy: 100.00%
4: Training accuracy: 100.00%
5: Training accuracy: 100.00%
6: Training accuracy: 100.00%
7: Training accuracy: 100.00%
8: Training accuracy: 100.00%
9: Training accuracy: 100.00%
10: Training accuracy: 100.00%
11: Training accuracy: 100.00%
12: Training accuracy: 100.00%
13: Training accuracy: 100.00%
14: Training accuracy: 100.00%
15: Training accuracy: 100.00%
16: Training accuracy: 100.00%
17: Training accuracy: 100.00%
18: Training accuracy: 100.00%
19: Training accuracy: 100.00%
20: Training accuracy: 100.00%


In [280]:
# --- Main execution ---

# Load dataset
dataset = [] # audio features
labels = [] # class ids

audio_files, classes = load_dataset("test")

# Prepare a dictionary of labels and their IDs
label_to_id = np.load("label_to_id.npy", allow_pickle=True).item()

# Extract features
for label_idx, class_name in enumerate(classes):
    for file_path in audio_files[class_name]:
        features = extract_audio_features(file_path, sr=8000)
        dataset.append(features)
        labels.append(label_to_id[class_name])


# Normalize feature-wise (0–1 scaling)
dataset = np.array(dataset)
dataset = (dataset - dataset.min(axis=0)) / (dataset.max(axis=0) - dataset.min(axis=0))

# generate codebooks
D = 10000  # Hypervector dimensionality
LEVELS = 256 # quantization levels
np.random.seed(42)
codebook = np.load("codebook.npy", allow_pickle=True).item()
value_level_hvs = np.load("value_level_hvs.npy", allow_pickle=True)

vectors = [] # encode dataset vectors

for features in dataset:
    encoded_hv = encode_feature_vector(features, codebook, value_level_hvs, D, LEVELS)
    vectors.append(encoded_hv)

vectors = np.array(vectors)
labels = np.array(labels)

# train the trained class hypervectors
class_hvs = np.load("class_hvs.npy", allow_pickle=True)

# Evaluate training performance
acc = evaluate_hd(vectors, labels, class_hvs)
print(f"Testing accuracy: {acc * 100:.2f}%")


Classes:  ['bell', 'glass', 'saw']
{'bell': ['audio/test/bell/1.wav'], 'glass': ['audio/test/glass/1.wav'], 'saw': ['audio/test/saw/1.wav']}
Testing accuracy: 66.67%


## Export model weights

# Inference

## Load sample

## Extract features

## HDC operations

## Model inference

In [ ]:
# query HV is compared to all the class HVs in the item memory using Hamming distance